# <header1>프로젝트 과제_4</header1>

### 이겸희_성균관대 

### 학습 준비단계

In [1]:
import pandas as pd
import numpy as np
import sys
import os
from matplotlib import colors
import random
from pathlib import Path
import imageio

import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from keras.models import load_model
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
import cv2

print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.21.5
TensorFlow Version :2.9.1
Matplotlib Version :3.5.3


In [2]:
IMAGE_PATH = "../유형별 두피 이미지/Training/원천_합친/"
JSON_PATH = "../유형별 두피 이미지/Training/라벨/"
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 3
IMG_SIZE = 64
RANDOM_STATE = 2018
TEST_SIZE = 0.2
VAL_SIZE = 0.2
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
PATIENCE = 5
VERBOSE = 1

In [3]:
plt.rc("font", family='Malgun Gothic')
plt.rc('axes', unicode_minus = False)

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'This pattern has match groups')
warnings.filterwarnings('ignore', 'The iterable function was deprecated in Matplotlib')

In [5]:
print(os.getcwd())

C:\Users\User\Desktop\솦트 인재 개발\5주차\과제4


### 데이터 가져오기

In [6]:
train_label = os.listdir(JSON_PATH)
train_image = os.listdir(IMAGE_PATH)
train_image

['0013_A2LEBJJDE00060O_1602578303771_2_TH.jpg',
 '0013_A2LEBJJDE00060O_1602578303771_3_TH.jpg',
 '0013_A2LEBJJDE00060O_1602578303771_4_LH.jpg',
 '0013_A2LEBJJDE00060O_1602578303771_5_RH.jpg',
 '0013_A2LEBJJDE00060O_1602578303771_6_BH.jpg',
 '0013_A2LEBJJDE00060O_1602911366731_2_TH.jpg',
 '0013_A2LEBJJDE00060O_1602911366731_3_TH.jpg',
 '0013_A2LEBJJDE00060O_1602911366731_4_LH.jpg',
 '0013_A2LEBJJDE00060O_1602911366731_5_RH.jpg',
 '0013_A2LEBJJDE00060O_1602911366731_6_BH.jpg',
 '0013_A2LEBJJDE00060O_1603096385510_2_TH.jpg',
 '0013_A2LEBJJDE00060O_1603096385510_4_LH.jpg',
 '0013_A2LEBJJDE00060O_1603096790519_2_TH.jpg',
 '0013_A2LEBJJDE00060O_1603096790519_3_TH.jpg',
 '0013_A2LEBJJDE00060O_1603096790519_4_LH.jpg',
 '0013_A2LEBJJDE00060O_1603096790519_5_RH.jpg',
 '0013_A2LEBJJDE00060O_1603096790519_6_BH.jpg',
 '0013_A2LEBJJDE00060O_1603257373184_2_TH.jpg',
 '0013_A2LEBJJDE00060O_1603257373184_3_TH.jpg',
 '0013_A2LEBJJDE00060O_1603257373184_4_LH.jpg',
 '0013_A2LEBJJDE00060O_1603257373184_5_R

In [7]:
df1 = pd.DataFrame(columns=['이미지 이름', '모낭사이홍반', '모낭홍반농포', '미세각질', '비듬', '탈모', '피지과다'])
df1

,이미지 이름,모낭사이홍반,모낭홍반농포,미세각질,비듬,탈모,피지과다


In [8]:
import json
import os
from tqdm import tqdm

In [9]:
with open(JSON_PATH + "[라벨]모낭사이홍반_0.양호/0013_A2LEBJJDE00060O_1606550825417_3_TH.json", "r", encoding="utf8") as f: 
    contents = f.read()
    json_data = json.loads(contents)
list(json_data.values())

['0013_A2LEBJJDE00060O_1606550825417_3',
 '0013_A2LEBJJDE00060O_1606550825417_3_TH.jpg',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [77]:
json_list = []
for di in train_label:
    file_name = os.listdir(JSON_PATH + di)
    for i in file_name:        
        with open(JSON_PATH + di + '/' + i, "r", encoding="utf8") as f: 
            contents = f.read()
            json_data = json.loads(contents)
            json_list.append(list(json_data.values()))

In [161]:
a = np.array(json_list)
data = pd.DataFrame(a, columns=['이미지 id', '이미지 이름', '모낭사이홍반', '모낭홍반농포', '미세각질', '비듬', '탈모', '피지과다'])
df_shuffled = sklearn.utils.shuffle(data)
df_shuffled

,이미지 id,이미지 이름,모낭사이홍반,모낭홍반농포,미세각질,비듬,탈모,피지과다
116125,2404_A2LEBJJDE000175_1604741072998_4,2404_A2LEBJJDE000175_1604741072998_4_LH.jpg,0,2,2,0,0,0
36408,1142_A2LEBJJDE00024K_1602473911141_5,1142_A2LEBJJDE00024K_1602473911141_5_RH.jpg,1,0,1,0,1,0
85611,1407_A2LEBJJDE001258_1602992609398_5,1407_A2LEBJJDE001258_1602992609398_5_RH.jpg,0,1,2,0,0,0
56805,5438_A2LEBJJDE00159W_1606266670197_2,5438_A2LEBJJDE00159W_1606266670197_2_TH.jpg,0,1,1,0,2,0
137341,6995_A2LEBJJDE00104R_1604643423115_5,6995_A2LEBJJDE00104R_1604643423115_5_RH.jpg,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...
23917,3937_A2LEBJJDE00046T_1605178711432_5,3937_A2LEBJJDE00046T_1605178711432_5_RH.jpg,1,0,2,0,0,0
105415,8281_A2LEBJJDE00055U_1602836672639_3,8281_A2LEBJJDE00055U_1602836672639_3_TH.jpg,0,1,1,0,2,2
15509,4162_A2LEBJJDE00013X_1604038847688_4,4162_A2LEBJJDE00013X_1604038847688_4_LH.jpg,0,1,3,0,0,1
29234,4919_A2LEBJJDE00064V_1604638156915_6,4919_A2LEBJJDE00064V_1604638156915_6_BH.jpg,2,0,3,0,0,0


In [162]:
df_3_item = df_shuffled[['이미지 이름', '모낭사이홍반', '미세각질', '탈모']]

In [177]:
data1 = df_3_item
data1

,이미지 이름,모낭사이홍반,미세각질,탈모
116125,2404_A2LEBJJDE000175_1604741072998_4_LH.jpg,0,2,0
36408,1142_A2LEBJJDE00024K_1602473911141_5_RH.jpg,1,1,1
85611,1407_A2LEBJJDE001258_1602992609398_5_RH.jpg,0,2,0
56805,5438_A2LEBJJDE00159W_1606266670197_2_TH.jpg,0,1,2
137341,6995_A2LEBJJDE00104R_1604643423115_5_RH.jpg,0,0,0
...,...,...,...,...
23917,3937_A2LEBJJDE00046T_1605178711432_5_RH.jpg,1,2,0
105415,8281_A2LEBJJDE00055U_1602836672639_3_TH.jpg,0,1,2
15509,4162_A2LEBJJDE00013X_1604038847688_4_LH.jpg,0,3,0
29234,4919_A2LEBJJDE00064V_1604638156915_6_BH.jpg,2,3,0


In [178]:
data1.set_index('이미지 이름',inplace=True)
data1 = data1.astype('float')   #inplace 가 없음
data1.reset_index(inplace=True)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138467 entries, 0 to 138466
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   이미지 이름  138467 non-null  object 
 1   모낭사이홍반  138467 non-null  float64
 2   미세각질    138467 non-null  float64
 3   탈모      138467 non-null  float64
dtypes: float64(3), object(1)
memory usage: 4.2+ MB


### 데이터 분류 및 학습준비

In [179]:
# , stratify=data['낭사이홍반', '모낭홍반농포', '미세각질', '비듬', '탈모', '피지과다']
train_df, test_df = train_test_split(data1, test_size=0.2, random_state=42)

In [180]:
# , stratify=train_df['낭사이홍반', '모낭홍반농포', '미세각질', '비듬', '탈모', '피지과다']
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [184]:
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

Train set rows: 88618
Test  set rows: 27694
Val   set rows: 22155


In [ ]:
for i in data1['이미지 이름']:
    try:
        image = skimage.io.imread(IMAGE_PATH + i)
    except:
        continue
    else: 
        idx = data1[data1['이미지 이름'] == i].index
        data1.drop(idx , inplace=True)

In [ ]:
data1

In [168]:
from PIL import Image

In [169]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (64, 64), mode='reflect')
    return image[:,:,:3]

# def read_image(file_name):
#     try:
#         image = skimage.io.imread(IMAGE_PATH + file_name)
#     except:
#         return None
#     else: 
# #         image = Image.open(IMAGE_PATH + file_name)
# #         image = image.convert("RGB")
# #         image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
# #         image = np.array(image)
#         image = skimage.io.imread(IMAGE_PATH + file_name)
#         image = skimage.transform.resize(image, (64, 64), mode='reflect')
#     return image[:,:,:3]

In [170]:
def categories_encoder(dataset):
    X = np.stack(dataset['이미지 이름'].apply(read_image))
    y = dataset[['모낭사이홍반', '미세각질',  '탈모']]
    return X, y

# dataset[['모낭사이홍반', '모낭홍반농포', '미세각질', '비듬', '탈모', '피지과다']]

In [171]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

64
64
64


### 모델 생성

In [58]:
n_class = 3
model = tf.keras.models.Sequential()
model.add(tf.keras.applications.densenet.DenseNet121(
    weights="imagenet", 
    include_top=False, 
    input_shape=(64, 64, 3)
    ))
model.add(tf.keras.layers.Flatten(name='flatten'))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(y_train.columns.size, activation='softmax'))

model.summary()
# model = tf.keras.models.Sequential()
# # conv 1
# model.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)))
# model.add(tf.keras.layers.Dropout(rate=0.5))                          #  DO1
# model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# # conv 2
# model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
# model.add(tf.keras.layers.Dropout(rate=0.5))                          # DO2
# model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# # conv 3
# model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
# model.add(tf.keras.layers.Dropout(rate=0.5))                           # DO3
# model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# # dense layers
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(3, activation='softmax'))

# model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 2, 2, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              4195328   
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 11,240,003
Trainable params: 11,154,307
Non-trainable params: 85,696
_____________________________________

In [59]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
image_generator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=180,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True,
        vertical_flip=True)
image_generator.fit(X_train)

### 모델 학습

In [63]:
%%time 
#한 번만 수행하지만 CPU 수행 시간과 실제 시간을 같이 출력합니다. %%time

from keras.callbacks import EarlyStopping 

early_stopping = EarlyStopping(patience = 2)
train_model1 = model.fit(X_train, y_train,
                         epochs=5,
                         validation_data=(X_val, y_val))

# train_model1  = model.fit_generator(image_generator.flow(X_train, y_train),
#                         epochs=5,
#                         validation_data=[X_val, y_val],
#                         callbacks = [early_stopping])

Epoch 1/5
20/20 [==============================] - 16s 449ms/step - loss: 14.1076 - accuracy: 0.4859 - val_loss: 363.5876 - val_accuracy: 0.6062
Epoch 2/5
20/20 [==============================] - 9s 453ms/step - loss: 12.6472 - accuracy: 0.5391 - val_loss: 266.9913 - val_accuracy: 0.2500
Epoch 3/5
20/20 [==============================] - 9s 443ms/step - loss: 10.9196 - accuracy: 0.6141 - val_loss: 341.0208 - val_accuracy: 0.1813
Epoch 4/5
20/20 [==============================] - 9s 457ms/step - loss: 11.8708 - accuracy: 0.5672 - val_loss: 113.6265 - val_accuracy: 0.1813
Epoch 5/5
 8/20 [===========>..................] - ETA: 5s - loss: 10.7437 - accuracy: 0.5547

KeyboardInterrupt: 

In [64]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 60.41020584106445
Test accuracy: 0.26499998569488525


In [ ]:
loss = train_model1.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.title('Loss')
plt.plot(epochs, train_model1.history['loss'], 'b', label='train_loss')
plt.plot(epochs, train_model1.history['val_loss'], 'g', label='val_loss')

plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')

plt.subplot(1, 2, 2)
plt.title('Accuray')
plt.plot(epochs, train_model1.history['accuracy'], 'b', label='train_accuracy')
plt.plot(epochs, train_model1.history['val_accuracy'], 'g', label='val_accuracy')

plt.grid(True)
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

In [ ]:
def test_accuracy_report(model):
    predicted = model.predict(X_test)
    test_predicted = np.argmax(predicted, axis=1)
    test_truth = np.argmax(y_test.values, axis=1)
    print(metrics.classification_report(test_truth, test_predicted, target_names=y_test.columns)) 
    test_res = model.evaluate(X_test, y_test.values, verbose=0)
    print('Loss function: %s, accuracy:' % test_res[0], test_res[1])

In [ ]:
test_accuracy_report(model)

In [ ]:
# Look at confusion matrix 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred, axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 